In [ ]:
from bs4 import BeautifulSoup
import requests
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import os
import getpass
os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter API key for Mistral: ")
api = os.environ["MISTRAL_API_KEY"]
from langchain_mistralai import ChatMistralAI

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def tavily_top_links(query, api_key, num_links=5):
    tavily_endpoint = 'https://api.tavily.com/search' 
    payload = {'query': query, 'max_results': num_links, 'api_key': api_key}
    resp = requests.post(tavily_endpoint, json=payload)
    data = resp.json()
    return [item['url'] for item in data.get('results', [])[:num_links]]

In [ ]:
def embed_chunks(chunks):
    return model.encode(chunks)

In [ ]:
def build_faiss_index(all_chunks):
    embeddings = embed_chunks(all_chunks)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings))
    return index, embeddings

In [ ]:
def retrieve_relevant_chunks(query, all_chunks, embeddings_index, embedder, N=5):
    query_vec = embedder.encode([query])
    D, I = embeddings_index.search(np.array(query_vec), k=N)
    return [all_chunks[i] for i in I[0]]

In [ ]:
def scrape_and_chunk(url, chunk_size=500):
    try:
        resp = requests.get(url, timeout=5)
        soup = BeautifulSoup(resp.text, 'html.parser')
        text = ' '.join([p.get_text() for p in soup.find_all('p')])
        chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
        return chunks
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return []

In [ ]:
def ask_rag_agent(query, context_chunks):
    context = "\n".join(context_chunks)
    prompt = f"Based only on the following:\n{context}\nAnswer this question: {query}"
    llm = ChatMistralAI(model="mistral-small-latest", api_key=api)
    messages = [
        {"role": "system", "content": "You are a helpful assistant that answers using ONLY the provided web context."},
        {"role": "user", "content": prompt}
    ]
    return llm.invoke(messages)  


In [ ]:
def search_scrape_rag(query, tavily_key):
    urls = tavily_top_links(query, tavily_key, num_links=5)
    all_chunks = []
    for url in urls:
        all_chunks.extend(scrape_and_chunk(url))
    index, _ = build_faiss_index(all_chunks)
    relevant_chunks = retrieve_relevant_chunks(query, all_chunks, index, model, k=5)
    answer = ask_rag_agent(query, relevant_chunks)
    return answer

In [ ]:
user_query = "What are the latest advances in the field of LLMs?"
tavily_key = getpass.getpass("Enter tavily key:")
result = search_scrape_rag(user_query, tavily_key= tavily_key)
print(result)

Error scraping https://www.mckinsey.com/capabilities/mckinsey-digital/our-insights/the-year-of-quantum-from-concept-to-reality-in-2025: HTTPSConnectionPool(host='www.mckinsey.com', port=443): Read timed out. (read timeout=5)
content="Based on the provided context, the latest advances in quantum computing include:\n\n1. **Real-world implementations**: Quantum computing is moving beyond theoretical exploration and entering a phase of practical innovation and application. This includes advancements in hardware, algorithms, and integration with other technologies, impacting fields such as artificial intelligence, pharmaceuticals, and telecommunications.\n\n2. **IBM's advancements**: IBM has achieved significant milestones and innovations that redefine the capabilities and applications of quantum computing. Their recent unveiling of new quantum processors, such as Heron and Condor, demonstrates the growing practicality of quantum technologies.\n\n3. **Quantum connectivity**: Advances in lin